In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

import pandas as pd

In [2]:
df = pd.read_excel("./advising.xlsx")

In [3]:
df

,Subject,URL,Text
0,Math,https://www.math.arizona.edu/academics/undergr...,The University of Arizona Search Search ABOUT....
1,Math,https://www.math.arizona.edu/academics/undergr...,The University of Arizona Search Search ABOUT....
2,Math,https://www.math.arizona.edu/academics/placement,The University of Arizona.........Search\t... ...
3,Math,https://www.math.arizona.edu/academics/undergr...,The University of Arizona.........Search\t... ...
4,Computer Science,https://cs.arizona.edu/undergraduate/undergrad...,Skip to main content The University of Arizona...
5,Computer Science,https://cs.arizona.edu/undergraduate/bachelor-...,Skip to main content The University of Arizona...
6,Computer Science,https://cs.arizona.edu/undergraduate/bachelor-...,Skip to main content The University of Arizona...
7,Computer Science,https://cs.arizona.edu/undergraduate/advising,Skip to main content The University of Arizona...
8,Computer Science,https://cs.arizona.edu/undergraduate/advising,Skip to main content The University of Arizona...
9,Computer Science,https://cs.arizona.edu/undergraduate/undergrad...,Skip to main content The University of Arizona...


In [4]:
df["Subject"]

0                Math
1                Math
2                Math
3                Math
4    Computer Science
5    Computer Science
6    Computer Science
7    Computer Science
8    Computer Science
9    Computer Science
Name: Subject, dtype: object

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [6]:
chroma_client = chromadb.PersistentClient(path="./stored_docs_db")
collection = chroma_client.create_collection(name="langchain")

In [7]:
def get_dicts(df: pd.DataFrame) -> list:
    dicts = []

    for i in df.index:
        dictionary = {}

        for col in df.loc[i].index:
            dictionary[col] = df.loc[i, col]
        dicts.append(dictionary)
    return dicts

get_dicts(df)[0].keys()

dict_keys(['Subject', 'URL', 'Text'])

In [8]:
len(get_dicts(df))

10

In [9]:
dicts = get_dicts(df)

for d in dicts:
    text = d["Text"]
    metadatas = {key : d[key] for key in ["Subject", "URL"]}
    
    documents = text_splitter.create_documents([text])
    contents = [doc.page_content for doc in documents] # list comprehension
    
    for j, content in enumerate(contents):
        collection.add(
            documents=content,
            metadatas=metadatas,
            ids=[d["Subject"] + "-" + str(j)]
        )

Insert of existing embedding ID: Math-0
Add of existing embedding ID: Math-0
Insert of existing embedding ID: Math-1
Add of existing embedding ID: Math-1
Insert of existing embedding ID: Math-2
Add of existing embedding ID: Math-2
Insert of existing embedding ID: Math-3
Add of existing embedding ID: Math-3
Insert of existing embedding ID: Math-4
Add of existing embedding ID: Math-4
Insert of existing embedding ID: Math-5
Add of existing embedding ID: Math-5
Insert of existing embedding ID: Math-0
Add of existing embedding ID: Math-0
Insert of existing embedding ID: Math-1
Add of existing embedding ID: Math-1
Insert of existing embedding ID: Math-2
Add of existing embedding ID: Math-2
Insert of existing embedding ID: Math-3
Add of existing embedding ID: Math-3
Insert of existing embedding ID: Math-4
Add of existing embedding ID: Math-4
Insert of existing embedding ID: Math-5
Add of existing embedding ID: Math-5
Insert of existing embedding ID: Math-0
Add of existing embedding ID: Math-0